In [62]:
import numpy as np
import pandas as pd

## Тесты

In [63]:
# Тест 0
A0 = np.array([[0, 2, 3], [1, 2, 4], [4, 5, 6]], dtype='float64')
b0 = np.array([13, 17, 32], dtype='float64')

# Тест 1
A1 = np.array([[8, 1, 1], [1, 10, 1], [1, 1, 12]], dtype='float64')
b1 = np.array([10, 12, 14], dtype='float64')

# Тест 2
A2 = np.array([[-8, 1, 1], [1, -10, 1], [1, 1, -12]], dtype='float64')
b2 = np.array([-10, -12, -14], dtype='float64')

# Тест 3
A3 = np.array([[-8, 9, 10], [11, -10, 7], [10, 11, -12]], dtype='float64')
b3 = np.array([10, 12, 14], dtype='float64')

# Тест 4
A4 = np.array([[8, 7, 7], [7, 10, 7], [7, 7, 12]], dtype='float64')
b4 = np.array([10, 12, 14], dtype='float64')


all_A = [A0, A1, A2, A3, A4]
all_b = [b0, b1, b2, b3, b4]


# Создание матрицы для пятого теста
def bad(n, eps):
    X = np.eye(n) # матрица слева
    Y = np.ones([n, n]) # матрица справа
    for i in range(n):
        for j in range(n):
            if i < j:
                X[i, j] = -1
                Y[i, j] = -1

    A = matrix_sum(X, eps * 6 * Y)
    b = -1 *np.ones(n)
    b[-1] = 1
    return A, b

## Дополнительные функции

In [64]:
def matrix_sum(A, B):
    sum = np.zeros(A.shape)
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            sum[i, j] = A[i,j] + B[i,j]
    return sum

def matrix_mult(A, B):
    if len(B.shape) == 1:
        mult = np.zeros(A.shape[0])
        for i in range(mult.shape[0]):
            mult[i] = sum(A[i] * B)
        return mult
    else:
        mult = np.zeros((A.shape[0],  B.shape[1]))
        for i in range(mult.shape[0]):
            for j in range(mult.shape[1]):
                mult[i, j] = sum(A[i] * B[:, j])
        return mult
    
def matrix_T(A):
    T = np.zeros((A.shape[1], A.shape[0]))
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            T[i,j] = A[j,i]
    return T

def mysqrt(num, eps = 0.00001):
    w0 = num / 2 
    w1 = 0.5 * (w0 + num / w0)
    while abs(w1 - w0) > eps:
        w0 = w1
        w1 = 0.5 * (w0 + num / w0)
    return w1

def minor(A,i,j):
    matr = A.tolist()
    matr = matr[:i] + matr[i+1:]
    for p in range(2):
        matr[p] = matr[p][:j] + matr[p][j+1:]
    return np.array(matr)

def det(A, n):
    if n == 1:
        return A[0][0]
    if n == 2:
        return A[0][0] * A[1][1] - A[0][1] * A[1][0]
    res = 0
    for i in range(n):
        mnr = minor(A, 0, i)
        res += ((-1)**i) * A[0][i] * det(mnr, n-1)
    return res  

## Нормы векторов и матриц

In [65]:
def vec_norm_1(vec):
    return sum(abs(vec))

def vec_norm_2(vec):
    return mysqrt(sum(vec**2))

def vec_norm_inf(vec):
    return max(abs(vec))

def vec_make_norms(vec):
    return np.array([vec_norm_1(vec), vec_norm_2(vec), vec_norm_inf(vec)])
    
    

def matr_norm_1(A):
    return max(sum(abs(A)))

def matr_norm_inf(A):
    return max(sum(abs(matrix_T(A))))

def matr_make_norms(A):
    return np.array([matr_norm_1(A), matr_norm_inf(A)]) 

## Метод простой итерации

In [66]:
def simple_iter(A, b, eps, norm_type):
    flag = 0
    A_copy = np.copy(A)
    b_copy = np.copy(b)
    norm_A = matr_make_norms(A)[norm_type]
    mu = 1 / norm_A
    B = matrix_sum(np.eye(A.shape[0]), -mu * A)
    norm_B = matr_make_norms(B)[norm_type]
    
    # Тут проверяем на сходимость метода
    if norm_B >= 1:
        A_copy = matrix_mult(matrix_T(A), A)
        b_copy = matrix_mult(matrix_T(A), b)
        norm_A = matr_make_norms(A_copy)[norm_type]
        mu = 1 / norm_A
        B = matrix_sum(np.eye(A.shape[0]), -mu * A_copy)
        norm_B = matr_make_norms(B)[norm_type]
        if norm_B >= 1:
            flag = True
    
    x0 = mu * b_copy
    x1 = matrix_mult(B, x0) + x0
    iterations = 1
    if flag:
        while vec_make_norms(matrix_mult(A_copy, x1) - b_copy)[norm_type] > eps:
            iterations += 1
            x0 = x1
            x1 = matrix_mult(B, x0) + (mu * b_copy)
        return x1, iterations
    
    while (norm_B / (1 - norm_B)) * vec_make_norms(x1 - x0)[norm_type] > eps:
        iterations += 1
        x0 = x1
        x1 = matrix_mult(B, x0) + (mu * b_copy)
    return x1, iterations

In [67]:
# Пример из методички
# Получен верный ответ

A = np.array([[3, 1, 1], [1, 5, 1], [1, 1, 7]])
b = np.array([5, 7, 9])

simple_iter(A, b, 0.001, 1)

(array([0.99975756, 1.00008312, 1.0000355 ]), 22)

In [68]:
all_eps = [10**(-3), 10**(-4), 10**(-5)]
all_A = [A0, A1, A2, A3, A4]
all_b = [b0, b1, b2, b3, b4]

# Эти списки буду заполнять данными, чтобы потом передать их в таблицу
tests = [] 
py_solution = []
eps = []
my_solution = []
error = []
num_iter = []

for ind_test in range(5):
    for ind_norm in range(2):
        tests.append(ind_test)
        pysolut = np.linalg.solve(all_A[ind_test], all_b[ind_test])
        py_solution.append(pysolut)
        eps.append(all_eps[ind_norm])
        solution = simple_iter(all_A[ind_test], all_b[ind_test], all_eps[ind_norm], ind_norm)
        my_solution.append(solution[0])
        error.append(abs(solution[0] - pysolut))
        num_iter.append(solution[1])


data = {'Номер теста': tests, 
        'Точное решение': py_solution,
        'Допустимая погрешность': eps,
        'Полученное решение': my_solution,
        'Абсолютная погрешность': error,
        'Количество итераций': num_iter}

pd.DataFrame(data)

,Номер теста,Точное решение,Допустимая погрешность,Полученное решение,Абсолютная погрешность,Количество итераций
0,0,"[1.0, 2.0, 3.0]",0.0010,"[0.999845094271208, 2.0006435387030934, 2.9996...","[0.0001549057287919542, 0.0006435387030934159,...",301
1,0,"[1.0, 2.0, 3.0]",0.0001,"[0.9998864682337695, 2.00025138768662, 2.99987...","[0.00011353176623052352, 0.0002513876866201414...",672
2,1,"[1.0, 1.0, 1.0]",0.0010,"[0.9997937362632765, 1.0000666209918094, 1.000...","[0.00020626373672349985, 6.66209918094296e-05,...",9
3,1,"[1.0, 1.0, 1.0]",0.0001,"[0.9999558036392794, 1.0000147700302364, 1.000...","[4.419636072061639e-05, 1.477003023642176e-05,...",11
4,2,"[1.6163793103448274, 1.5043103448275863, 1.426...",0.0010,"[1.6161698933539395, 1.5042117756699733, 1.426...","[0.00020941699088794508, 9.856915761297103e-05...",27
5,2,"[1.6163793103448274, 1.5043103448275863, 1.426...",0.0001,"[1.6163388966965422, 1.5042913227926276, 1.426...","[4.0413648285220205e-05, 1.9022034958737777e-0...",32
6,3,"[1.4462540716612375, 1.1661237785016283, 1.107...",0.0010,"[1.4460675845369866, 1.1659560796544155, 1.107...","[0.00018648712425095582, 0.0001676988472127938...",48
7,3,"[1.4462540716612375, 1.1661237785016283, 1.107...",0.0001,"[1.4462231236485543, 1.1660959484513163, 1.107...","[3.094801268321845e-05, 2.783005031203878e-05,...",58
8,4,"[-0.02272727272727272, 0.6590909090909091, 0.7...",0.0010,"[-0.022560897252084988, 0.6589837026146577, 0....","[0.00016637547518773277, 0.0001072064762513980...",1563
9,4,"[-0.02272727272727272, 0.6590909090909091, 0.7...",0.0001,"[-0.022701540906900464, 0.6590743284152637, 0....","[2.5731820372257297e-05, 1.6580675645361254e-0...",1934


### пятый тест, простая итерация

In [69]:
all_e = [10**(-4), 10**(-5), 10**(-6)]
all_dimensions = [4, 5, 6] # увеличивающаяся размерность матрицы
all_eps = [10**(-3), 10**(-4), 10**(-5)]

dimension = [] 
e = []
py_solution = []
eps = []
my_solution = []
error = []
num_iter = []

for ind_dim in range(3):
    n = all_dimensions[ind_dim]
    current_e = all_e[ind_dim]
    A, b = bad(n, current_e) # Создание матрицы для 5-го теста
    for ind_norm in range(2):                                                
        dimension.append(n)
        e.append(current_e)
        true_solution = np.linalg.solve(A, b)
        py_solution.append(true_solution)
        eps.append(all_eps[ind_norm])
        solution = simple_iter(A, b, all_eps[ind_norm], ind_norm)        
        my_solution.append(solution[0])
        error.append(abs(solution[0] - true_solution))
        num_iter.append(solution[1])

data = {'Размерность': dimension, 
        'e': e,
        'Точное решение': py_solution,
        'Допустимая погрешность': eps,
        'Полученное решение': my_solution,
        'Абсолютная погрешность': error,
        'Количество итераций': num_iter}

pd.DataFrame(data)

,Размерность,e,Точное решение,Допустимая погрешность,Полученное решение,Абсолютная погрешность,Количество итераций
0,4,0.000100,"[4.5660461435365003e-17, -1.598119937751854e-1...",0.0010,"[-0.01500217032640666, -0.007624239295972143, ...","[0.015002170326406706, 0.0076242392959721276, ...",386
1,4,0.000100,"[4.5660461435365003e-17, -1.598119937751854e-1...",0.0001,"[-0.002516058234747709, -0.001278683660226944,...","[0.0025160582347477545, 0.001278683660226928, ...",702
2,5,0.000010,"[3.372716650852626e-15, 1.7350318559187908e-15...",0.0010,"[-0.05786543034126448, -0.02905670959640618, -...","[0.05786543034126785, 0.029056709596407916, 0....",183
3,5,0.000010,"[3.372716650852626e-15, 1.7350318559187908e-15...",0.0001,"[-0.009901638439270735, -0.004972036515787137,...","[0.009901638439274107, 0.004972036515788872, 0...",2014
4,6,0.000001,"[6.874950230562001e-15, 3.4500899717070323e-15...",0.0010,"[-0.03472934795729679, -0.017388891066318354, ...","[0.03472934795730367, 0.017388891066321802, 0....",38
5,6,0.000001,"[6.874950230562001e-15, 3.4500899717070323e-15...",0.0001,"[-0.034664511949708945, -0.017351044720285796,...","[0.03466451194971582, 0.017351044720289244, 0....",50


## Метод Зейделя

In [70]:
def Zeidel(A, b, eps, norm_type):
    
    #дополнительные функции для метода Зейделя
    def make_C_d(A, b):
        C = np.zeros(A.shape)
        d = np.zeros(b.shape)
        for i in range(A.shape[0]):
            d[i] = b[i] / A[i,i] # заполняем d
            for j in range(A.shape[1]):
                if i == j:
                    C[i, j] = 0
                else:
                    C[i, j] = -A[i, j] / A[i,i]  # заполняем C
        return C, d


    def make_x1(C, d, x0):
        x1 = np.copy(d)
        for i in range(C.shape[0]):
            for j in range(C.shape[0]):
                if j > i: # у маши +1
                    x1[i] += C[i,j] * x0[j]
                else:
                    x1[i] += C[i,j] * x1[j]
        return x1
    
    def diag_dominance(A):
        D = np.diag(np.abs(A))
        S = np.sum(np.abs(A), axis=1) - D 
        if np.all(D > S):
            return True
        else:
            return False
    
    #########################################
    if diag_dominance(A):
        C, d = make_C_d(A, b)
        x0 = np.copy(d)
        x1 = make_x1(C, d, x0)
        iterations = 1
        while vec_make_norms(matrix_mult(A, x1) - b)[norm_type] > eps:
            iterations += 1
            x0 = x1
            x1 = make_x1(C, d, x0)
        return x1, iterations
    else:
        A_copy = matrix_mult(matrix_T(A), A)
        b_copy = matrix_mult(matrix_T(A), b)
        C, d = make_C_d(A_copy, b_copy)
        x0 = np.copy(d)
        x1 = make_x1(C, d, x0)
        iterations = 1
        while vec_make_norms(matrix_mult(A_copy, x1) - b_copy)[norm_type] > eps:
            iterations += 1
            x0 = x1
            x1 = make_x1(C, d, x0)
        return x1, iterations

In [71]:
# Пример из методички
# Получен верный ответ

A = np.array([[3, 1, 1], [1, 5, 1], [1, 1, 7]])
b = np.array([5, 7, 9])

Zeidel(A, b, 0.001, 1)

(array([1.00022736, 0.99997049, 0.99997174]), 4)

In [72]:
all_eps = [10**(-3), 10**(-4), 10**(-5)]
all_A = [A0, A1, A2, A3, A4]
all_b = [b0, b1, b2, b3, b4]

# Эти списки буду заполнять данными, чтобы потом передать их в таблицу
tests = [] 
py_solution = []
eps = []
my_solution = []
error = []
num_iter = []

for ind_test in range(5):
    for ind_norm in range(2):
        tests.append(ind_test)
        pysolut = np.linalg.solve(all_A[ind_test], all_b[ind_test])
        py_solution.append(pysolut)
        eps.append(all_eps[ind_norm])
        solution = Zeidel(all_A[ind_test], all_b[ind_test], all_eps[ind_norm], ind_norm)
        my_solution.append(solution[0])
        error.append(abs(solution[0] - pysolut))
        num_iter.append(solution[1])

data = {'Номер теста': tests, 
        'Точное решение': py_solution,
        'Допустимая погрешность': eps,
        'Полученное решение': my_solution,
        'Абсолютная погрешность': error,
        'Количество итераций': num_iter}

pd.DataFrame(data)

,Номер теста,Точное решение,Допустимая погрешность,Полученное решение,Абсолютная погрешность,Количество итераций
0,0,"[1.0, 2.0, 3.0]",0.0010,"[1.0008244107253077, 1.9982460882450823, 3.000...","[0.0008244107253077004, 0.0017539117549176808,...",347
1,0,"[1.0, 2.0, 3.0]",0.0001,"[1.000106108328696, 1.9997742573704098, 3.0001...","[0.00010610832869595299, 0.0002257426295901865...",458
2,1,"[1.0, 1.0, 1.0]",0.0010,"[1.0000751455801504, 0.9999952668366608, 0.999...","[7.514558015042638e-05, 4.7331633391545225e-06...",3
3,1,"[1.0, 1.0, 1.0]",0.0001,"[1.0000013251080924, 1.0000004542593308, 0.999...","[1.3251080923648573e-06, 4.5425933081766345e-0...",4
4,2,"[1.6163793103448274, 1.5043103448275863, 1.426...",0.0010,"[1.6163660738580021, 1.5043063467619153, 1.426...","[1.323648682527434e-05, 3.998065670973716e-06,...",4
5,2,"[1.6163793103448274, 1.5043103448275863, 1.426...",0.0001,"[1.61637863106003, 1.5043101332778357, 1.42672...","[6.792847973002836e-07, 2.1154975060611036e-07...",5
6,3,"[1.4462540716612375, 1.1661237785016283, 1.107...",0.0010,"[1.4462513768694611, 1.1661217146528595, 1.107...","[2.6947917763919804e-06, 2.063848768774079e-06...",19
7,3,"[1.4462540716612375, 1.1661237785016283, 1.107...",0.0001,"[1.446253732780663, 1.1661235189645975, 1.1074...","[3.3888057449082964e-07, 2.5953703075565215e-0...",22
8,4,"[-0.02272727272727272, 0.6590909090909091, 0.7...",0.0010,"[-0.022826656977123383, 0.6591381928218435, 0....","[9.938424985066197e-05, 4.7283730934433166e-05...",235
9,4,"[-0.02272727272727272, 0.6590909090909091, 0.7...",0.0001,"[-0.022740375861296958, 0.6590971431274293, 0....","[1.3103134024236573e-05, 6.234036520225139e-06...",284


## Пятый тест, Зейдель

In [73]:
all_e = [10**(-3), 10**(-4), 10**(-5)]
all_dimensions = [4, 5, 6] # увеличивающаяся размерность матрицы
all_eps = [10**(-3), 10**(-4), 10**(-5)] # для сетода простой итерации

dimension = [] 
e = []
py_solution = []
eps = []
my_solution = []
error = []
num_iter = []

for ind_dim in range(3):
    n = all_dimensions[ind_dim]
    current_e = all_e[ind_dim]
    A, b = bad(n, current_e) # Создание матрицы для 5-го теста
    for ind_norm in range(2):
        dimension.append(n)
        e.append(current_e)
        true_solution = np.linalg.solve(A, b)
        py_solution.append(true_solution)
        eps.append(all_eps[ind_norm])
        solution = Zeidel(A, b, all_eps[ind_norm], ind_norm)
        my_solution.append(solution[0])
        error.append(abs(solution[0] - true_solution))
        num_iter.append(solution[1])

data = {'Размерность': dimension, 
        'e': e,
        'Точное решение': py_solution,
        'Допустимая погрешность': eps,
        'Полученное решение': my_solution,
        'Абсолютная погрешность': error,
        'Количество итераций': num_iter}

pd.DataFrame(data)

,Размерность,e,Точное решение,Допустимая погрешность,Полученное решение,Абсолютная погрешность,Количество итераций
0,4,0.00100,"[-3.914198728742427e-16, -2.1449272876017223e-...",0.0010,"[0.014367833475526504, 0.007128135782147363, 0...","[0.014367833475526896, 0.0071281357821475775, ...",58
1,4,0.00100,"[-3.914198728742427e-16, -2.1449272876017223e-...",0.0001,"[0.0017018816516902735, 0.0008443335259318871,...","[0.001701881651690665, 0.0008443335259321015, ...",96
2,5,0.00010,"[-7.892183643334199e-16, -4.3317029880225616e-...",0.0010,"[0.052625091066103136, 0.026294147702571356, 0...","[0.05262509106610393, 0.02629414770257179, 0.0...",204
3,5,0.00010,"[-7.892183643334199e-16, -4.3317029880225616e-...",0.0001,"[0.007239712539295007, 0.0036173252525732317, ...","[0.007239712539295796, 0.003617325252573665, 0...",354
4,6,0.00001,"[6.76862304732345e-15, 3.4328831830745736e-15,...",0.0010,"[0.18739888103056201, 0.09369332487351861, 0.0...","[0.18739888103055524, 0.09369332487351519, 0.0...",544
5,6,0.00001,"[6.76862304732345e-15, 3.4328831830745736e-15,...",0.0001,"[0.028448419798782765, 0.014223281503537369, 0...","[0.028448419798775996, 0.014223281503533936, 0...",1118


## Метод Гаусса (LU – разложение)

In [74]:
def gaus_back(A, b, mode = 1): # mode 1 для верхнетреугольной матрицы, 0 - для нижней
    n = A.shape[0]
    x = np.zeros(n)
    m = np.concatenate((A, b.reshape(n,1)), axis=1) # делаем расширенную матрицу
    if mode:
        for k in range(n - 1, -1, -1):
            x[k] = (m[k, -1] - sum([m[k, j] * x[j] for j in range(k + 1, n)])) / m[k, k]
        return x
    else:
        for k in range(0, n, 1):
            x[k] = (m[k, -1] - sum([m[k, j] * x[j] for j in range(k - 1, -1, -1)])) / m[k, k]
        return x

def LU(A, b):
    U = np.copy(A)
    b_cop = np.copy(b)
    if U[0, 0] == 0:
        for i in range(1, A.shape[0]):
            if U[i, 0] != 0:
                U[[0, i]] = U[[i, 0]]
                b_cop[0], b_cop[i] = b_cop[i], b_cop[0]
                break
    L = np.eye(A.shape[0])
    for i in range(A.shape[0]):
        for j in range(i+1, A.shape[0]):
            x = U[j, i] / U[i,i]
            L[j, i] = x
            U[j] = U[j] - (x * U[i])
    Y = gaus_back(L, b_cop, mode = 0)
    X = gaus_back(U, Y)
    return X

In [75]:
tests = [] 
py_solution = []
my_solution = []
error = []

for ind_test in range(5):
    tests.append(ind_test)
    pysolut = np.linalg.solve(all_A[ind_test], all_b[ind_test])
    py_solution.append(pysolut)
    solution = LU(all_A[ind_test], all_b[ind_test]) # мое решение
    my_solution.append(solution)
    error.append(abs(solution - pysolut)) # погрешность
    
data = {'Номер теста': tests, 
        'Точное решение': py_solution,
        'Полученное решение': my_solution,
        'Абсолютная погрешность': error}

pd.DataFrame(data)

,Номер теста,Точное решение,Полученное решение,Абсолютная погрешность
0,0,"[1.0, 2.0, 3.0]","[1.0, 2.0, 3.0]","[0.0, 0.0, 0.0]"
1,1,"[1.0, 1.0, 1.0]","[1.0, 1.0, 0.9999999999999999]","[0.0, 0.0, 1.1102230246251565e-16]"
2,2,"[1.6163793103448274, 1.5043103448275863, 1.426...","[1.6163793103448276, 1.504310344827586, 1.4267...","[2.220446049250313e-16, 2.220446049250313e-16,..."
3,3,"[1.4462540716612375, 1.1661237785016283, 1.107...","[1.4462540716612375, 1.1661237785016283, 1.107...","[0.0, 0.0, 2.220446049250313e-16]"
4,4,"[-0.02272727272727272, 0.6590909090909091, 0.7...","[-0.022727272727272707, 0.6590909090909092, 0....","[1.3877787807814457e-17, 1.1102230246251565e-1..."


## Пятый тест, LU

In [76]:
all_e = [10**(-4), 10**(-5), 10**(-6)]
all_dimensions = [4, 5, 6] # увеличивающаяся размерность матрицы
all_eps = [10**(-3), 10**(-4), 10**(-5)]

dimension = [] 
e = []
py_solution = []
my_solution = []
error = []

for ind_dim in range(3):
    n = all_dimensions[ind_dim]
    current_e = all_e[ind_dim]
    A, b = bad(n, current_e) # Создание матрицы для 5-го теста
    for ind_eps in range(3):
        dimension.append(n)
        e.append(current_e)
        true_solution = np.linalg.solve(A, b)
        py_solution.append(true_solution)
        solution = LU(A, b)
        my_solution.append(solution)
        error.append(abs(solution - true_solution))
        
data = {'Размерность': dimension, 
        'e': e,
        'Точное решение': py_solution,
        'Полученное решение': my_solution,
        'Абсолютная погрешность': error}

pd.DataFrame(data)

,Размерность,e,Точное решение,Полученное решение,Абсолютная погрешность
0,4,0.000100,"[4.5660461435365003e-17, -1.598119937751854e-1...","[0.0, 0.0, 0.0, 0.9994003597841297]","[4.5660461435365003e-17, 1.598119937751854e-17..."
1,4,0.000100,"[4.5660461435365003e-17, -1.598119937751854e-1...","[0.0, 0.0, 0.0, 0.9994003597841297]","[4.5660461435365003e-17, 1.598119937751854e-17..."
2,4,0.000100,"[4.5660461435365003e-17, -1.598119937751854e-1...","[0.0, 0.0, 0.0, 0.9994003597841297]","[4.5660461435365003e-17, 1.598119937751854e-17..."
3,5,0.000010,"[3.372716650852626e-15, 1.7350318559187908e-15...","[1.7762502643843875e-15, 8.88071851077996e-16,...","[1.5964663864682385e-15, 8.469600048407948e-16..."
4,5,0.000010,"[3.372716650852626e-15, 1.7350318559187908e-15...","[1.7762502643843875e-15, 8.88071851077996e-16,...","[1.5964663864682385e-15, 8.469600048407948e-16..."
5,5,0.000010,"[3.372716650852626e-15, 1.7350318559187908e-15...","[1.7762502643843875e-15, 8.88071851077996e-16,...","[1.5964663864682385e-15, 8.469600048407948e-16..."
6,6,0.000001,"[6.874950230562001e-15, 3.4500899717070323e-15...","[4.440865453307907e-15, 2.2204194042174626e-15...","[2.4340847772540945e-15, 1.2296705674895697e-1..."
7,6,0.000001,"[6.874950230562001e-15, 3.4500899717070323e-15...","[4.440865453307907e-15, 2.2204194042174626e-15...","[2.4340847772540945e-15, 1.2296705674895697e-1..."
8,6,0.000001,"[6.874950230562001e-15, 3.4500899717070323e-15...","[4.440865453307907e-15, 2.2204194042174626e-15...","[2.4340847772540945e-15, 1.2296705674895697e-1..."


## QR - разложение

In [77]:
from functools import reduce


def QR(A, b):
    shape = A.shape[0]
    Qs = []
    Qs.append(np.eye(shape)) # тут все Q, изначально только Q0
    Rs = []
    Rs.append(np.copy(A)) # Тут все R
    sub_R = np.copy(A)
    length = A.shape[0] # нужен, чтобы следить за размерностью орта и тд
    
    # первый шаг вне цикла:
    y = sub_R[:, 0]
    z = np.eye(length)[0]
    alf = vec_norm_2(y)
    ro = vec_norm_2(y -  alf*z)
    w = (y -  alf*z) / ro
    
    Qs.append(matrix_sum(np.eye(length), -2 * matrix_mult(w.reshape(length,1), w.reshape(1,length))))
    Rs.append(matrix_mult(Qs[1], Rs[0]))
    
    sub_R = Rs[1][1:,1:] #срез
    length -= 1 
    
    for i in range(1, shape-1):
        
        y = sub_R[:, 0]
        z = np.eye(length)[0] # орт
        alf = vec_norm_2(y)
        ro = vec_norm_2(y -  alf*z)
        w = (y -  alf*z) / ro
        
        new_Q = np.eye(shape)
        new_Q[i:, i:] = matrix_sum(np.eye(length), -2 * matrix_mult(w.reshape(length,1), w.reshape(1,length)))
        Qs.append(new_Q)
        
        new_R = np.copy(Rs[i])
        new_R[i:, i:] = matrix_mult(new_Q[i:, i:], Rs[i][i:, i:])
        Rs.append(new_R)
        
        sub_R = new_R[i+1:, i+1:]
        length -= 1
        
    # получили разложение QR
    final_Q = reduce(lambda x, y: matrix_mult(x, y), Qs) # получили разложение QR 
    final_R = Rs[-1]
    
    # рассматриваем модифицированную СЛАУ
    Q_T = matrix_T(final_Q)
    Y = matrix_mult(Q_T, b)
    X = gaus_back(final_R, Y)
    return X

In [78]:
# Тест из методички
# Верно
aa = np.array([[3, 1, 1], [1, 5, 1], [1, 1, 7]], dtype = 'float64')
bb = np.array([5, 7, 9], dtype = 'float64')

QR(aa, bb)

array([1., 1., 1.])

In [79]:
tests = [] 
py_solution = []
my_solution = []
error = []

for ind_test in range(5):
    tests.append(ind_test)
    pysolut = np.linalg.solve(all_A[ind_test], all_b[ind_test])
    py_solution.append(pysolut)
    solution = QR(all_A[ind_test], all_b[ind_test]) # мое решение
    my_solution.append(solution)
    error.append(abs(solution - pysolut)) # погрешность
    
data = {'Номер теста': tests, 
        'Точное решение': py_solution,
        'Полученное решение': my_solution,
        'Абсолютная погрешность': error}

pd.DataFrame(data)

,Номер теста,Точное решение,Полученное решение,Абсолютная погрешность
0,0,"[1.0, 2.0, 3.0]","[1.000000000007502, 1.999999999977479, 3.00000...","[7.501999021997108e-12, 2.2521096099126225e-11..."
1,1,"[1.0, 1.0, 1.0]","[1.0000000000001734, 0.9999999999996853, 0.999...","[1.7341683644644945e-13, 3.147482274812319e-13..."
2,2,"[1.6163793103448274, 1.5043103448275863, 1.426...","[1.6163793103448212, 1.5043103448275728, 1.426...","[6.217248937900877e-15, 1.354472090042691e-14,..."
3,3,"[1.4462540716612375, 1.1661237785016283, 1.107...","[1.4462540716613885, 1.1661237785018024, 1.107...","[1.509903313490213e-13, 1.7408297026122455e-13..."
4,4,"[-0.02272727272727272, 0.6590909090909091, 0.7...","[-0.022727272727261293, 0.659090909090908, 0.7...","[1.1428358259735205e-14, 1.1102230246251565e-1..."


## Пятый тест, QR

In [61]:
all_e = [10**(-4), 10**(-5), 10**(-6)]
all_dimensions = [4, 5, 6] # увеличивающаяся размерность матрицы
all_eps = [10**(-3), 10**(-4), 10**(-5)] # для сетода простой итерации

dimension = [] 
e = []
py_solution = []
my_solution = []
error = []

for ind_dim in range(3):
    n = all_dimensions[ind_dim]
    current_e = all_e[ind_dim]
    A, b = bad(n, current_e) # Создание матрицы для 5-го теста
    for ind_eps in range(3):
        dimension.append(n)
        e.append(current_e)
        true_solution = np.linalg.solve(A, b)
        py_solution.append(true_solution)
        solution = QR(A, b)
        my_solution.append(solution)
        error.append(abs(solution - true_solution))
        
data = {'Размерность': dimension, 
        'e': e,
        'Точное решение': py_solution,
        'Полученное решение': my_solution,
        'Абсолютная погрешность': error}

pd.DataFrame(data)

,Размерность,e,Точное решение,Полученное решение,Абсолютная погрешность
0,4,0.000100,"[4.5660461435365003e-17, -1.598119937751854e-1...","[2.2191133836179925e-16, 2.2177821615846485e-1...","[1.7625087692643424e-16, 2.377594155359834e-16..."
1,4,0.000100,"[4.5660461435365003e-17, -1.598119937751854e-1...","[2.2191133836179925e-16, 2.2177821615846485e-1...","[1.7625087692643424e-16, 2.377594155359834e-16..."
2,4,0.000100,"[4.5660461435365003e-17, -1.598119937751854e-1...","[2.2191133836179925e-16, 2.2177821615846485e-1...","[1.7625087692643424e-16, 2.377594155359834e-16..."
3,5,0.000010,"[3.372716650852626e-15, 1.7350318559187908e-15...","[1.7762502516165572e-15, 8.887538687785948e-16...","[1.5964663992360688e-15, 8.46277987140196e-16,..."
4,5,0.000010,"[3.372716650852626e-15, 1.7350318559187908e-15...","[1.7762502516165572e-15, 8.887538687785948e-16...","[1.5964663992360688e-15, 8.46277987140196e-16,..."
5,5,0.000010,"[3.372716650852626e-15, 1.7350318559187908e-15...","[1.7762502516165572e-15, 8.887538687785948e-16...","[1.5964663992360688e-15, 8.46277987140196e-16,..."
6,6,0.000001,"[6.874950230562001e-15, 3.4500899717070323e-15...","[-9.325817451201298e-15, -4.628818040137215e-1...","[1.62007676817633e-14, 8.078908011844248e-15, ..."
7,6,0.000001,"[6.874950230562001e-15, 3.4500899717070323e-15...","[-9.325817451201298e-15, -4.628818040137215e-1...","[1.62007676817633e-14, 8.078908011844248e-15, ..."
8,6,0.000001,"[6.874950230562001e-15, 3.4500899717070323e-15...","[-9.325817451201298e-15, -4.628818040137215e-1...","[1.62007676817633e-14, 8.078908011844248e-15, ..."
